In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import datapackage
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import statsmodels.api as sm
import statsmodels.tsa.vector_ar.vecm as vecm

## testing armington elastcities UK gas market ##

url = 'https://raw.githubusercontent.com/joe-ascroft/phd/master/data/UKlng.csv'
df = pd.read_csv(url)
df["Date"] = pd.to_datetime(df["Date"],dayfirst=True)
for col in df.columns[1:]:
    df[col] = pd.to_numeric(df[col],errors='coerce')

In [2]:
print(df)

         Date       LNG   Domestic      lnMD        NBP        HH    lnPdPm
0  2009-03-01  13422.80  198753.92 -2.695113   6.544974  4.577455  0.357554
1  2009-06-01  19025.23  181093.51 -2.253248   4.275921  3.701228  0.144335
2  2009-09-01  30321.50  129951.46 -1.455304   3.826394  3.164407  0.189957
3  2009-12-01  49467.99  169545.34 -1.231795   5.196570  4.355763  0.176499
4  2010-03-01  48753.68  178842.46 -1.299725   5.230260  5.126481  0.020041
5  2010-06-01  49206.12  169541.99 -1.237083   5.526589  4.305862  0.249593
6  2010-09-01  44496.09  140433.57 -1.149333   6.595826  4.288814  0.430427
7  2010-12-01  64390.11  153696.67 -0.870021   8.173289  3.802931  0.765099
8  2011-03-01  79545.14  146728.71 -0.612261   9.087511  4.192500  0.773604
9  2011-06-01  82736.41  127397.62 -0.431653   9.468383  4.365517  0.774221
10 2011-09-01  60813.75  101190.31 -0.509187   9.152192  4.120508  0.798017
11 2011-12-01  51698.93  136035.52 -0.967479   9.661959  3.325536  1.066566
12 2012-03-0

In [6]:
lag_order1 = vecm.select_order(data=df[['lnPdPm', 'lnMD']], maxlags=5, deterministic="ci")
print(lag_order1.selected_orders)

rank = vecm.select_coint_rank(endog=df[['lnPdPm', 'lnMD']],
                              k_ar_diff=5,
                              det_order=0)

print(rank)

{'aic': 0, 'bic': 0, 'hqic': 0, 'fpe': 0}
Johansen cointegration test using trace test statistic with 5% significance level
r_0 r_1 test statistic critical value
-------------------------------------
  0   2          16.85          15.49
  1   2          6.564          3.841
-------------------------------------


In [7]:
m1 = vecm.VECM(df[['lnPdPm', 'lnMD']], deterministic="ci", k_ar_diff=1, coint_rank=1)  
vecm_res_1 = m1.fit()
vecm_res_1.summary()

,coef,std err,z,P>|z|,[0.025,0.975]
L1.lnPdPm,0.0574,0.151,0.381,0.703,-0.238,0.353
L1.lnMD,-0.0031,0.059,-0.052,0.959,-0.119,0.113
,coef,std err,z,P>|z|,[0.025,0.975]
L1.lnPdPm,0.1648,0.357,0.461,0.645,-0.536,0.865
L1.lnMD,-0.0986,0.140,-0.705,0.481,-0.373,0.176
,coef,std err,z,P>|z|,[0.025,0.975]
ec1,-0.0053,0.023,-0.230,0.818,-0.051,0.040
,coef,std err,z,P>|z|,[0.025,0.975]
ec1,-0.1914,0.055,-3.493,0.000,-0.299,-0.084
,coef,std err,z,P>|z|,[0.025,0.975]


In [8]:
granger_results = vecm_res_1.test_granger_causality(caused="lnMD", signif=0.05)
granger_results.summary()

Test statistic,Critical value,p-value,df
0.7536,3.120,0.474,"(2, 74)"
